# [문제] Managing Conversation History2
- trim_messages()
- RunnablePassthrough 
- itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation) 
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [ ]:
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import trim_messages
from langchain_core.output_parsers import StrOutputParser

from operator import itemgetter
from dotenv import load_dotenv


load_dotenv()

True

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-4o",             
)

prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 이력서 작성 컨설턴트입니다. 아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성하세요. 문장은 자연스럽고 매끄럽게 작성합니다'),
    ('placeholder', '{chat_history}'),
    ('human', '{query}')
])

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

trimmer = trim_messages(
    max_tokens=65,
    strategy='last',
    token_counter=llm,
    include_system=True,
    start_on="human",    
)

wrapped_chain = RunnableWithMessageHistory(
    (
        RunnablePassthrough.assign(chat_history=itemgetter("chat_history") | trimmer) 
        | prompt 
        | llm 
        | StrOutputParser()
    ),
    get_session_history,
    history_messages_key="chat_history",
    input_messages_key="query",
)

In [20]:
while True : 
    query = input("이름, 경력, 스킬 등 정보를 입력하세요(S 입력시 종료) >>")
    if query.upper() == "S":
        break

    aimessage = wrapped_chain.invoke(
        {'query' : query},
        config={'configurable': {'session_id': '박보검'}}
    )
    print(aimessage)


박보검

**경력 요약**
12년 넘게 무대에서 다채로운 퍼포먼스를 선보이며 전 세계 관객들을 매료시켰습니다. 현대적 감각과 에너지를 무대 위에서 발산하며, 특히 록킹 춤에 대한 깊은 이해와 연출로 높은 평가를 받고 있습니다. 다양한 무대 경험과 열정을 바탕으로 지금도 끊임없이 발전하며 새로운 도전에 도전하고 있습니다.

**주요 스킬**
- **춤:** 강력하고 역동적인 퍼포먼스를 자랑하며, 다양한 춤 스타일을 소화할 수 있는 능력을 갖추고 있습니다.
- **록킹:** 이 분야의 전문가로서, 록킹의 전형적인 요소를 완벽하게 소화하며 무대에서 탁월한 에너지를 펼칩니다.
  
**경험**
- **주요 공연과 페스티벌 참가:** 전 세계 주요 무대와 페스티벌에서의 공연 경험이 풍부합니다. 다양한 관객층을 상대로 관객들의 호응을 이끌어내는 데 탁월한 능력을 발휘했습니다.
- **안무 참여 및 지휘:** 여러 프로젝트에서 안무에 직접 참여하거나 팀을 이끌어 안무를 완성하는 데 기여했습니다. 이를 통해 팀워크와 리더십을 발휘하며 뛰어난 공연 퀄리티를 유지했습니다.

**교육 및 기타 활동**
- 다수의 워크샵과 배틀 참여를 통해 끊임없이 다른 스타일을 탐구하며 춤 실력을 연마하였습니다.
- 춤 분야에서의 다양한 협업과 콜라보레이션을 통해 창의적인 안무를 기획하고 실현하였습니다.

박보검은 무대 위에서 관객과 하나가 되는 순간을 소중히 여기며, 춤을 통해 세상에 긍정적인 에너지를 전파하고 있습니다. 춤에 대한 열정과 지속적인 학습으로 언제나 최상의 퍼포먼스를 선보이겠습니다.
[이름] 김하늘  
[연락처] 010-1234-5678  
[이메일] haneul.kim@email.com  
[주소] 서울특별시 강남구 행복로 123  

---

**프로필**  
열정적이고 창의적인 공연 기획자로서 다년간의 경험을 바탕으로 다양한 문화 예술 이벤트를 성공적으로 이끌어온 전문가입니다. 지속 가능한 관객 참여와 혁신적인 기획을 통해 관객에게 특별한 경험을 제공하고자 합니다.

**경

In [21]:
store['박보검']

InMemoryChatMessageHistory(messages=[HumanMessage(content='이름은 박보검, 경력은 12년, 스킬은 춤추는거야. 롹킹을 특히 잘해. ', additional_kwargs={}, response_metadata={}), AIMessage(content='박보검\n\n**경력 요약**\n12년 넘게 무대에서 다채로운 퍼포먼스를 선보이며 전 세계 관객들을 매료시켰습니다. 현대적 감각과 에너지를 무대 위에서 발산하며, 특히 록킹 춤에 대한 깊은 이해와 연출로 높은 평가를 받고 있습니다. 다양한 무대 경험과 열정을 바탕으로 지금도 끊임없이 발전하며 새로운 도전에 도전하고 있습니다.\n\n**주요 스킬**\n- **춤:** 강력하고 역동적인 퍼포먼스를 자랑하며, 다양한 춤 스타일을 소화할 수 있는 능력을 갖추고 있습니다.\n- **록킹:** 이 분야의 전문가로서, 록킹의 전형적인 요소를 완벽하게 소화하며 무대에서 탁월한 에너지를 펼칩니다.\n  \n**경험**\n- **주요 공연과 페스티벌 참가:** 전 세계 주요 무대와 페스티벌에서의 공연 경험이 풍부합니다. 다양한 관객층을 상대로 관객들의 호응을 이끌어내는 데 탁월한 능력을 발휘했습니다.\n- **안무 참여 및 지휘:** 여러 프로젝트에서 안무에 직접 참여하거나 팀을 이끌어 안무를 완성하는 데 기여했습니다. 이를 통해 팀워크와 리더십을 발휘하며 뛰어난 공연 퀄리티를 유지했습니다.\n\n**교육 및 기타 활동**\n- 다수의 워크샵과 배틀 참여를 통해 끊임없이 다른 스타일을 탐구하며 춤 실력을 연마하였습니다.\n- 춤 분야에서의 다양한 협업과 콜라보레이션을 통해 창의적인 안무를 기획하고 실현하였습니다.\n\n박보검은 무대 위에서 관객과 하나가 되는 순간을 소중히 여기며, 춤을 통해 세상에 긍정적인 에너지를 전파하고 있습니다. 춤에 대한 열정과 지속적인 학습으로 언제나 최상의 퍼포먼스를 선보이겠습니다.', additional_kwargs={}, response_metadata=

In [28]:
for r in store['박보검'].messages: 
    print(f"[{r.type.upper()}]: {r.content}")

[HUMAN]: 이름은 박보검, 경력은 12년, 스킬은 춤추는거야. 롹킹을 특히 잘해. 
[AI]: 박보검

**경력 요약**
12년 넘게 무대에서 다채로운 퍼포먼스를 선보이며 전 세계 관객들을 매료시켰습니다. 현대적 감각과 에너지를 무대 위에서 발산하며, 특히 록킹 춤에 대한 깊은 이해와 연출로 높은 평가를 받고 있습니다. 다양한 무대 경험과 열정을 바탕으로 지금도 끊임없이 발전하며 새로운 도전에 도전하고 있습니다.

**주요 스킬**
- **춤:** 강력하고 역동적인 퍼포먼스를 자랑하며, 다양한 춤 스타일을 소화할 수 있는 능력을 갖추고 있습니다.
- **록킹:** 이 분야의 전문가로서, 록킹의 전형적인 요소를 완벽하게 소화하며 무대에서 탁월한 에너지를 펼칩니다.
  
**경험**
- **주요 공연과 페스티벌 참가:** 전 세계 주요 무대와 페스티벌에서의 공연 경험이 풍부합니다. 다양한 관객층을 상대로 관객들의 호응을 이끌어내는 데 탁월한 능력을 발휘했습니다.
- **안무 참여 및 지휘:** 여러 프로젝트에서 안무에 직접 참여하거나 팀을 이끌어 안무를 완성하는 데 기여했습니다. 이를 통해 팀워크와 리더십을 발휘하며 뛰어난 공연 퀄리티를 유지했습니다.

**교육 및 기타 활동**
- 다수의 워크샵과 배틀 참여를 통해 끊임없이 다른 스타일을 탐구하며 춤 실력을 연마하였습니다.
- 춤 분야에서의 다양한 협업과 콜라보레이션을 통해 창의적인 안무를 기획하고 실현하였습니다.

박보검은 무대 위에서 관객과 하나가 되는 순간을 소중히 여기며, 춤을 통해 세상에 긍정적인 에너지를 전파하고 있습니다. 춤에 대한 열정과 지속적인 학습으로 언제나 최상의 퍼포먼스를 선보이겠습니다.
[HUMAN]: 공연과 페스티벌 명을 가짜로 넣어줘. 
[AI]: [이름] 김하늘  
[연락처] 010-1234-5678  
[이메일] haneul.kim@email.com  
[주소] 서울특별시 강남구 행복로 123  

---

**프로필**  
열정적이고 창의적인 공연 기획자로서 다년간의 경험을 